In [21]:
import pandas as pd
import requests

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
open('./image-predictions.tsv', 'wb').write(r.content)
images_df = pd.read_csv('./image-predictions.tsv', sep='\t')
archive_df = pd.read_csv('./WeRateDogs/twitter-archive-enhanced.csv')
attr_df = pd.read_csv('./WeRateDogs/attributes.csv')

In [28]:
# By "unsorting" a list I wanted to do something like this:

dfs = ['a', 'b', 'c']
dflist = ['B', 'A', 'C']
a, b, c = [y for y,_ in sorted(zip(dflist, dfs))][0], [y for y,_ in sorted(zip(dflist, dfs))][1], [y for y,_ in sorted(zip(dflist, dfs))][2]
print(a, b, c)

A B C


In [22]:
# This function synchronizes the three dataframes by tweet_id,
# so that they all contain the same set and sequence of tweet_ids.
# This could be done more simply with a join, but I want to keep 
# all three separate to examine separately as I clean


def synch_dfs(archive, attributes, images):
    
    # sorts the three dataframes by length so we can begin with the shortest
    dfs = [archive, attributes, images]
    dflist = sorted(dfs, key=lambda x:x.shape[0])
    
    # Creates masks and filters the dataframes by them
    m1 = dflist[1].tweet_id.isin(dflist[0].tweet_id)
    try:
        dflist[1] = dflist[1][m1].drop('Unnamed: 0', axis=1)
    except KeyError:
        dflist[1] = dflist[1][m1]
    m2 = dflist[2].tweet_id.isin(dflist[1].tweet_id)
    dflist[2] = dflist[2][m2]
    m3 = dflist[0].tweet_id.isin(dflist[1].tweet_id)   
    dflist[0] = dflist[0][m3]
    
    # Sorts all three dfs by tweet_id and resets their indices
    dflist[0] = dflist[0].sort_values(by=['tweet_id']).reset_index(drop=True)
    dflist[1] = dflist[1].sort_values(by=['tweet_id']).reset_index(drop=True)    
    dflist[2] = dflist[2].sort_values(by=['tweet_id']).reset_index(drop=True)
    
    # tests that they are synchronized and returns True, True if they are
    print(dflist[1]['tweet_id'].equals(dflist[0]['tweet_id']), dflist[2]['tweet_id'].equals(dflist[0]['tweet_id']))
    
    return dflist[0], dflist[1], dflist[2]

In [23]:
archive_df, attr_df, images_df = synch_dfs(archive_df, attr_df, images_df)

True True


In [27]:
archive_df

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2068,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2069,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2070,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2071,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
